In [112]:
import random
from hashlib import sha256
import codecs
# from Crypto.Hash import SHA256
# from Crypto.Cipher import AES

In [113]:
def FastModExpo(g,a,p):
    product = 1
    while a > 1.0:
        if a & 1:   
            product = (product * g)%p
        g = (g*g)%p
        a = a >> 1
    product = (product*g)%p
    return product

In [114]:
def get_bytes(a):
    size = a.bit_length()
    plus_one = 0
    if FastModExpo(size,1,8) > 0:
        plus_one = 1
    
    return size//8 + plus_one

1. Select a random a as your DH private key. Using the provided shared Diffie-Hellman (DH) parameters g and p, calculate and submit your public key g^a(mod p).

In [115]:
g = 5
p = 233000556327543348946447470779219175150430130236907257523476085501968599658761371268535640963004707302492862642690597042148035540759198167263992070601617519279204228564031769469422146187139698860509698350226540759311033166697559129871348428777658832731699421786638279199926610332604408923157248859637890960407

a = (1 << 127) | random.getrandbits(127)
print(a)
ga = FastModExpo(g,a,p)
print(ga)

2. Using the values provided by server, you will know calculate the shared key. To do so, you will need to calculate the following values:
    1. The server will provide you with a password and salt. Use these values to calculate the hashed password x = H(salt || password)^1000.
    2. Using the hashed password and the provided B value, calculate the server’s DH public key g^b - B -k * g^x(mod p).
    3. Calculate the values u = H(g^a || g^b) and k = H(p||g).
    4. Calculate the shared key as (g^b)^(a+u*x) (mod p).

In [122]:
salt = int('c41dac96',16)
size = get_bytes(salt)
salt = salt.to_bytes(size,'big')
print(salt)

pw = bytes('imperceivably','ascii')
to_hash = salt + pw
print(to_hash)

h = sha256(to_hash).digest()
print(h)
for i in range(999):
    h= sha256(h).digest()
print(h)

x_int = int.from_bytes(h,'big')
print(x_int)

b'\xc4\x1d\xac\x96'
b'\xc4\x1d\xac\x96imperceivably'
b'\x06l4Y\xb5|A\xb3\x86r\r>\x8dl\xa5\xb3Cv\xad\x93+\xb4\x16Mw\\80Js\x9d\x14'
b'\x8a\x7f F\x8a\x8d\x84<\x99\xd6\xc5\xc6N\x93\x1d\xd8\xb3\xa56\xd5 \xd4\xd2\xec\xb2T?\xde6\xa8\x18#'
62643785439388293186088327223109950130048329324384032426550715651489784141859


In [123]:
p_byte = p.to_bytes(get_bytes(p),'big')
g_byte = g.to_bytes(get_bytes(g),'big')
to_hash = p_byte + g_byte


h = sha256(to_hash).digest()
k_int = int.from_bytes(h,'big')
print(k_int)

93155903901601396783468346788441692422337325899474882765676037711917603533093


In [124]:
B_vec =  61917745973786954066293966877861468165591139049753603360733593275929098820816707648977157256295176983647265937427557877408268394750018997174233108828293256906962416400647272613664960311843279552754385119291866241029333733566885390453783748247094432443485347344664820938208622212336217607887076034265405292160
v = FastModExpo(g,x_int,p)

gb = B_vec - (k_int * v)
gb = FastModExpo(gb,1,p)
print(gb)

95621464991392076828567853346195776337713723783936756256617663058965267345857513589305257848238864187675643764356243898489337036051194512288869031974777229587062058523498320494364267213816741558982674826682094961575808877628296510291735565031548950422911859344187576442780087518975947988125339470526074128281


In [125]:
ga_byte = ga.to_bytes(get_bytes(ga),'big')
gb_byte = gb.to_bytes(get_bytes(gb),'big')
to_hash = ga_byte + gb_byte

h = sha256(to_hash).digest()
u_int = int.from_bytes(h,'big')
print(u_int)

49460703485336090870476194280311310245592964760156692241759124960095709404567


In [127]:
raise_by = a + (u_int * x_int)
shared_key = FastModExpo(gb,raise_by,p)
print(shared_key)

90410996302887961306264738995114326316653933587340845055565981419019592512601023470960740932950305000628034587762114477227720515139641915090297903793619230988119769494878599100344026898692711394121057239959039860158138109572285937235499401231150725149309029212040264617703217319893484430798101972424947583692


3. Calculate M1, the zero-knowledge proof that the Client (you) knows the password. M1 = H(H(p)) xor H(g) || H(netid) || salt || g^a || g^b || shared key)

In [168]:
p_hash = sha256(p_byte).digest() 
g_hash = sha256(g_byte).digest()

p_hash_int = int.from_bytes(p_hash,'big')
g_hash_int = int.from_bytes(g_hash,'big')

p_xor_g = p_hash_int ^ g_hash_int
p_xor_g_byte = p_xor_g.to_bytes(get_bytes(p_xor_g),'big')

netid = bytes('abahour2','ascii')
netid_hash = sha256(netid).digest()

shared_key_byte = shared_key.to_bytes(get_bytes(shared_key),'big')

# print(p_xor_g_byte , netid_hash , salt , ga_byte , ga_byte , shared_key)

to_hash = p_xor_g_byte + netid_hash + salt + ga_byte + gb_byte + shared_key_byte
M1 = sha256(to_hash).hexdigest()
print(M1)

d9b32d8ee07962f5a7fc138c75fed55dae32e58d7ebeab3ac0da278e1cadbaf3


4. Calculate M2 the zero-knowledge proof that the server knows the verifier (v = g^x(mod p)). M2 = H(g^a || M1 || shared key) 

In [169]:
M1_byte = int(M1,16)
M1_byte = M1_byte.to_bytes(get_bytes(M1_byte),'big')

print(ga_byte , M1_byte , shared_key_byte)

to_hash = ga_byte + M1_byte + shared_key_byte
M2 = sha256(to_hash).hexdigest()
print(M2)

b'\xa7.\xc5\x9c\xda0)\x923\x85\xf4\x85u\xa2\xbf\xac\xa0\x1a\xb9\xf1V\xef\xb6GKmQ\xecEpL;\xc2p>;b\x92\x8c\x81`\xabT\xf7A[\xf9)U{\xdc\xcf\xd8ZR4\xf3\xadf\x97\xc6\t\xde-\xfa\xc6\xb9r\x14\xfa\xf9\x05e\xe2\xca\xf9"\x95\xd9\x1c\xb8\x7f \xec\x1d\x18\xbe\xb0\xdbS4@O\x80\xc4@\xbb\x9fo\xad\xcf\xc2\x8d+VK/\xd5L\x95\x1e\xd9\xe7e%\xf8\x01\x0bw\xea\xb8J\r\x96\xde\xa7\xd9\x06' b"\xd9\xb3-\x8e\xe0yb\xf5\xa7\xfc\x13\x8cu\xfe\xd5]\xae2\xe5\x8d~\xbe\xab:\xc0\xda'\x8e\x1c\xad\xba\xf3" b'\x80\xbf\xe1[0\xac5Kl\xe8\x9c\x0f\x1e\xac\xc8M\xf2\x90/\xba\xa76\xd2|Y/\xd6\x04TD\xa8\xf5q\xb4Q5\xd9\xc0\xf6\\\xf9V\xebk\xc3\xda\xe2\x08\xcf\x7ft\x8b\x97\tm\x1f\xda\xe5X\x82%y\x827\x0e\xea\x9f\xe2\xc8*.\x1f\xdb\xf3\x02\x87N_\xbd\xf9\xae\xa39\x15\\\x9b\x8e[\xc0w\xf1\nVA0\xd7\x80s\x87\xce\xd1&\x18`\x98]\xde^)\x0b\xed\x0c!\xe8\xb1G\xa5\xe4\xce<U\xea}J|Z\xca\xcc'
734ac7fa9da7105a4c17c85aec662449d1b8606b6d071cef93bb06f718fee02e
